In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot

from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

### Getting the Data Set 

In [2]:
# Loading the market data set 

# Probably a less circuitous route to getting this done, my pd skills are rusty...

md = pd.read_csv('Tech_Eval/market_data.csv')
inter = md[['base_asset_id', 'price_open']]
mdbtc = inter[inter['base_asset_id'] == 'Bitcoin_BTC_BTC']
mdbtc_arr = mdbtc['price_open'].values

### Univariate Predictive Models

#### Autoregression

In [3]:
model = AR(mdbtc_arr)
model_fit = model.fit()
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

13179.25688155909


#### Moving-averages

In [4]:
model = ARMA(mdbtc_arr, order=(0, 1))
model_fit = model.fit(disp=False)

yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

10466.667170191871


#### Autoregressive Moving-averages

In [5]:
model = ARMA(mdbtc_arr, order=(2, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

13543.738537098749


#### Autoregressive Integrated Moving-averages

In [6]:
model = ARIMA(mdbtc_arr, order=(1, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr), typ='levels')
print(yhat[0])

13632.46129707051


#### Seasonal Autoregressive Integrated moving-average 

In [7]:
model = SARIMAX(mdbtc_arr, order=(1, 1, 1), seasonal_order=(1, 1, 1, 1))
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(mdbtc_arr), len(mdbtc_arr))
print(yhat[0])

13909.786725343305


### References

https://machinelearningmastery.com/time-series-forecasting-methods-in-python-cheat-sheet/